In [3]:
import cv2
import geopandas as gpd
import tifffile as tf
import numpy as np
import itertools
from osgeo import gdal
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.figure

In [6]:
# Author: Travis Simmons
# Date: 7/20/2020
# Objective: Creating a repository of black and white photos of dirt or lettuce for HAAR Cascade training.

#Imputting a utm referenced ortho
utm_ortho = r"C:\Users\Travis\Desktop\Results\Gantry Orthos\Lettuce_2.24.20_15m_Ortho.tif"

#Imputting the geojson for selection, you should have a geojson for false immages and one for positive
selection_boxes = gpd.read_file(r"C:\Users\Travis\Desktop\Results\season10_individual_lettuce.geojson")
#selection_boxes = gpd.read_file(r"C:\Users\Travis\Desktop\new_error\false_selection.geojson")

# What file do you want to store the pictures in?
outpath = "C:\\Users\\Travis\\Desktop\\Results\\Gantry Orthos\\p\\"

#Converting it to EPSG: 4326 and Opening it in gdal
input_raster = gdal.Open(utm_ortho)
output_raster = r"C:\Users\Travis\Desktop\new_error\gantry_EPSG_raster.tif"
gdal.Warp(output_raster,input_raster,dstSRS='EPSG:4326')
EPSG_ortho = r"C:\Users\Travis\Desktop\new_error\gantry_EPSG_raster.tif"
right_ortho = gdal.Open(EPSG_ortho)

#Looping throught the selection boxes
for index, row in selection_boxes.bounds.iterrows():
    
    minx = 	row.minx
    maxy = row.maxy
    maxx = row.maxx
    miny = row.miny
    
    # Change the "false" to "true" if you are making positives
    out = outpath+"true"+f'{index}'+".tif"
    
    
    #Cropping the ortho using the selection boxes
    gdal.Translate(out, right_ortho, projWin = [minx, maxy, maxx, miny])
    
    
    #Turning each image gray for HAAR cascade training
    img_gray = cv2.imread(out, cv2.IMREAD_GRAYSCALE)
    cv2.imwrite(out,img_gray)